### **5. Практическая реализация авторизации OAuth 2.0**

#### **Шаг 1: Регистрация клиента OAuth в панели администратора**

OAuth требует регистрации клиента (приложения), чтобы идентифицировать его при запросе токенов.

1. Запустите сервер:
   ```bash
   python manage.py runserver
   ```
2. Перейдите в админ-панель по адресу:
   ```
   http://127.0.0.1:8000/admin/
   ```
3. Авторизуйтесь с логином и паролем суперпользователя, который вы создали ранее.
4. В разделе **"Applications"** нажмите **"Add"**.
5. Заполните форму:
   - **Name:** Укажите название клиента (например, `Test Client`).
   - **Client ID:** Оставьте пустым, он будет создан автоматически.
   - **Client Secret:** Оставьте пустым, он также будет создан автоматически.
   - **Client Type:** Выберите **Confidential** (для защищённых приложений).
   - **Authorization Grant Type:** Выберите **Resource owner password-based** (пароль пользователя).
   - **Redirect URIs:** Оставьте пустым (не требуется для данного типа авторизации).

6. Сохраните приложение. После этого будет создан клиент с уникальными `Client ID` и `Client Secret`. Они понадобятся для получения токенов.


#### **Шаг 2: Получение токенов доступа через запросы**

Теперь вы можете запросить `Access Token` (токен доступа) для взаимодействия с API.

1. **Сценарий для получения токена через Postman или Python (без стороннего ПО):**

Создайте Python-скрипт `get_token.py` для получения токена:
```python
import requests

url = "http://127.0.0.1:8000/o/token/"
data = {
    "grant_type": "password",
    "username": "ваш_логин",  # Логин существующего пользователя
    "password": "ваш_пароль",  # Пароль существующего пользователя
    "client_id": "ваш_client_id",  # Укажите Client ID из админки
    "client_secret": "ваш_client_secret",  # Укажите Client Secret из админки
}

response = requests.post(url, data=data)

if response.status_code == 200:
    print("Access Token:", response.json().get("access_token"))
    print("Refresh Token:", response.json().get("refresh_token"))
else:
    print("Error:", response.json())
```

2. Запустите скрипт:
   ```bash
   python get_token.py
   ```
3. В результате вы получите:
   - **Access Token** (например, `abcdef12345...`) — используется для доступа к защищённым API.
   - **Refresh Token** (например, `12345xyz...`) — используется для обновления истёкшего `Access Token`.


#### **Шаг 3: Работа с Access Token для доступа к API**

Теперь используем `Access Token`, чтобы получить доступ к защищённому ресурсу.

1. Перейдите в браузер и откройте URL (замените `ACCESS_TOKEN` своим токеном):
   ```
   http://127.0.0.1:8000/api/protected/?access_token=ACCESS_TOKEN
   ```
2. Если всё настроено верно, вы увидите ответ:
   ```json
   {
       "message": "Вы успешно авторизовались!",
       "user": "ваше_имя_пользователя"
   }
   ```


#### **Шаг 4: Обновление токенов через Refresh Token**

`Access Token` имеет ограниченный срок действия (по умолчанию 3600 секунд). Чтобы не вводить логин и пароль заново, можно использовать `Refresh Token` для обновления.

Создайте Python-скрипт `refresh_token.py`:
```python
import requests

url = "http://127.0.0.1:8000/o/token/"
data = {
    "grant_type": "refresh_token",
    "refresh_token": "ваш_refresh_token",  # Укажите ранее полученный Refresh Token
    "client_id": "ваш_client_id",
    "client_secret": "ваш_client_secret",
}

response = requests.post(url, data=data)

if response.status_code == 200:
    print("New Access Token:", response.json().get("access_token"))
    print("New Refresh Token:", response.json().get("refresh_token"))
else:
    print("Error:", response.json())
```

Запустите скрипт:
```bash
python refresh_token.py
```

Вы получите новый `Access Token` и `Refresh Token`, которые можно использовать далее.

#### **Дополнительные рекомендации**



1. **Токены в админке:** Все токены можно увидеть в панели администратора в разделе **"Access Tokens"**.
2. **Срок действия токенов:** Если требуется изменить срок действия токенов, настройте это в `settings.py`:
   ```python
   OAUTH2_PROVIDER = {
       "ACCESS_TOKEN_EXPIRE_SECONDS": 3600,  # Время действия токена (в секундах)
       "REFRESH_TOKEN_EXPIRE_SECONDS": 86400,  # Время действия Refresh Token
   }
   ```
3. **Безопасность:**
   - Никогда не передавайте `Client Secret` в открытом виде.
   - Используйте защищённое соединение (HTTPS) на боевом сервере.